In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import linecache

Download "EThOS_CSV_202210.csv" from [British Library Research Repository](https://bl.iro.bl.uk/concern/datasets/480ce636-ab7c-4b80-8512-08e372dbf08a)

**To Do:**
- Copy over and adapt code to download script from Markus' notebook

In [7]:
csv_file = "../EThOS_CSV_202210.csv"
df = pd.read_csv(csv_file, encoding="ISO-8859-2")
df.head()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_26930/3604971026.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding="ISO-8859-2")


,Title,DOI,Author,Author ISNI,ORCID,Institution,Institution ISNI,Date,Qualification,Abstract,Subject Discipline,Supervisor(s),Funder(s),EThOS URL,IR URL
0,Computation and measurement of turbulent flow ...,,"Loizou, Panos A.",https://isni.org/isni/0000000136122593,,University of Manchester,https://isni.org/isni/0000000121662407,1989,Thesis (Ph.D.),,Physical Sciences,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,
1,Prolactin and growth hormone secretion in norm...,,"Prescott, R. W. G.",https://isni.org/isni/0000000134992122,,University of Newcastle upon Tyne,https://isni.org/isni/0000000104627212,1983,Thesis (Ph.D.),,Biological Sciences,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,
2,Influence of strain fields on flame propagation,,"Mendes-Lopes, J. M. C.",https://isni.org/isni/0000000133937105,,University of Cambridge,https://isni.org/isni/0000000121885934,1983,Thesis (Ph.D.),Flame propagation can be highly influenced by ...,Engineering & Technology,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,
3,"Connectivity, flow and transport in network mo...",,"Robinson, Peter Clive",https://isni.org/isni/0000000135292822,,University of Oxford,https://isni.org/isni/0000000419368948,1984,Thesis (Ph.D.),In order to evaluate the safety of radioactive...,Physical Sciences,"Rae, John",,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,http://ora.ox.ac.uk/objects/uuid:c61bf73b-b794...
4,The theory and implementation of a high qualit...,,"Lower, K. N.",https://isni.org/isni/0000000136133567,,University of Bristol,https://isni.org/isni/0000000419367603,1985,Thesis (Ph.D.),,Engineering & Technology,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,


In [8]:
len(df)

610535

In [9]:
df.columns

Index(['Title', 'DOI', 'Author', 'Author ISNI', 'ORCID', 'Institution',
       'Institution ISNI', 'Date', 'Qualification', 'Abstract',
       'Subject Discipline', 'Supervisor(s)', 'Funder(s)', 'EThOS URL',
       'IR URL'],
      dtype='object')

## Looking for NaNs

In [10]:
print(f"Number of rows with NaNs: {df.isnull().any(axis=1).sum()}")

Number of rows with NaNs: 3


which columns do these occur?

In [11]:
[f"{col}: {df[col].isnull().any()}" for col in df.columns]

['Title: False',
 'DOI: False',
 'Author: False',
 'Author ISNI: False',
 'ORCID: False',
 'Institution: False',
 'Institution ISNI: False',
 'Date: False',
 'Qualification: False',
 'Abstract: False',
 'Subject Discipline: False',
 'Supervisor(s): False',
 'Funder(s): True',
 'EThOS URL: False',
 'IR URL: True']

Looks like there's some in `Funder(s)` and `IR URL`...

## Fixing NaN in `IR URL` column

In [12]:
df[df["IR URL"].isnull()]

,Title,DOI,Author,Author ISNI,ORCID,Institution,Institution ISNI,Date,Qualification,Abstract,Subject Discipline,Supervisor(s),Funder(s),EThOS URL,IR URL
278145,Archaeological Museums and Schools : Teaching ...,"Arwa""",https://isni.org/isni/0000000426923620,,University of Newcastle upon Tyne,https://isni.org/isni/0000000104627212,2010,Thesis (Ph.D.),,Education,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,NaN
353875,Shall I say 'tis so?' : Elizabethan fictions a...,"J. K.""",,,University of Cambridge,https://isni.org/isni/0000000121885934,2010,Thesis (Ph.D.),This thesis concerns emergent representations ...,Language & Literature,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,NaN


In [13]:
# extracting the 278145th item
particular_line = linecache.getline('EThOS_CSV_202210.csv', 278145+2)
# print the particular line
print(particular_line)

In [14]:
# extracting the 353875th item
particular_line = linecache.getline('EThOS_CSV_202210.csv', 353875+2)
# print the particular line
print(particular_line)

Seems like the problem here is that there is an unclosed quote at the start and so it misses out a column. We can fix this with the following:

In [15]:
def fix_IR_URL_null(df):
    null_df = df[df["IR URL"].isnull()]
    for index, row in null_df.iterrows():
        new_row = []
        # Title
        new_row.append(row["Title"].split(",")[0])
        # DOI
        new_row.append(row["Title"].split(",")[1])
        # Author
        first_name = row["Title"].split(",")[2]
        second_name = row["DOI"][:-1]
        new_row.append(first_name + second_name)
        # Remaining columns
        colnames = [col for col in df.columns
                    if col not in ["Title", "DOI", "IR URL"]]
        new_row += row[colnames].tolist()
        df.loc[index] = new_row
    return df

df = fix_IR_URL_null(df)

## Fixing NaN in `Funder(s)` column

In [16]:
df[df["Funder(s)"].isnull()]

,Title,DOI,Author,Author ISNI,ORCID,Institution,Institution ISNI,Date,Qualification,Abstract,Subject Discipline,Supervisor(s),Funder(s),EThOS URL,IR URL
272375,Integrated non-invasive investigations of capt...,,"Rabineau, Johanna Romy Maria",https://isni.org/isni/0000000426855875,,University of Exeter,https://isni.org/isni/0000000419368024,2009,Thesis (Ph.D.),The prime aim of the present study was to inve...,Biological Sciences,"Brown, Anne",NaN,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,http://hdl.handle.net/10036/106776


In [17]:
# extracting the 272375th item
particular_line = linecache.getline('EThOS_CSV_202210.csv', 272375+2)
# print the particular line
print(particular_line)

Looks like `n/a` was put here, so we'll just fix this by replacing it with `' '` - which is the current convention in this dataset for missing data.

In [18]:
df.fillna(' ', inplace=True)

Just to double check that we've fixed the NaNs:

In [19]:
print(f"Number of rows with NaNs: {df.isnull().any(axis=1).sum()}")

Number of rows with NaNs: 0


In [20]:
df.isnull().any().any()

False

To do this in one go, the `remove_na()` function in `remove_na.py` does both of these steps in one go. Note that this saves a new cleaned df as a csv in your current directory by default.

In [21]:
from remove_na import remove_na
df = remove_na(df)

## Data missingness

In [22]:
for col in df.columns:
    print(f"proportion of missing {col}: {sum(df[col]==' ') / len(df)}")

proportion of missing Title: 0.0
proportion of missing DOI: 0.9121508185443914
proportion of missing Author: 0.0
proportion of missing Author ISNI: 0.21315731284856723
proportion of missing ORCID: 0.9863120050447558
proportion of missing Institution: 0.0
proportion of missing Institution ISNI: 0.0009974858116242311
proportion of missing Date: 6.551630946628777e-06
proportion of missing Qualification: 0.006892315755853473
proportion of missing Abstract: 0.40922305846511664
proportion of missing Subject Discipline: 0.06691508267339301
proportion of missing Supervisor(s): 0.7654581637416364
proportion of missing Funder(s): 0.9072764051200995
proportion of missing EThOS URL: 0.0
proportion of missing IR URL: 0.3886099895992859


- Every entry as `Title` and `Institution`
- Some missing Authors
- Some missing Dates
- Have Abstracts for about 60\% of entries
- Most entries have a subject discipline

In [23]:
df["Institution"].value_counts()

University of Oxford                                42036
University of Cambridge                             40043
University of Edinburgh                             29376
University College London (University of London)    27126
Imperial College London                             26575
                                                    ...  
Wessex Institute of Technology                          1
University of Wales, Newport                            1
Royal Agricultural University                           1
Trinity College, Bristol                                1
Arts University Bournemouth                             1
Name: Institution, Length: 159, dtype: int64

In [24]:
df["Subject Discipline"].value_counts()

Medicine & Health                             96561
Physical Sciences                             86681
Engineering & Technology                      76345
Social, Economic & Political Studies          69651
Biological Sciences                           61226
                                              40854
Philosophy, Psychology & Religious Studies    30129
Language & Literature                         26320
History & Archaeology                         18842
Business & Administrative Studies             18530
Education                                     18494
Computer Science                              14401
Mathematics & Statistics                      12692
Agriculture & Veterinary Sciences             11934
Law                                            7069
Creative Arts & Design                         5898
Architecture, Building & Planning              4806
Sport & Recreation                             4527
Music                                          3487
Librarianshi